# SNAP Recipients and Benefits by Gross Income (October 2025)

Analysis of SNAP distribution across income levels using PolicyEngine-US microsimulation.

In [ ]:
from policyengine_us import Microsimulation
import pandas as pd
import plotly.graph_objects as go
from policyengine.utils.charts import format_figure, COLOUR_SCHEMES

sim = Microsimulation()

## Cumulative Distribution by Income

This chart shows what percentage of SNAP recipients and benefits go to households below various income thresholds.

In [ ]:
# Calculate for recipients (person level)
snap_person = sim.calculate('snap', period='2025-10', map_to='person')
income_ratio_person = sim.calculate('snap_gross_income_fpg_ratio', period='2025-10', map_to='person')

# Filter to only SNAP recipients and create dataframe
df_person = pd.DataFrame({
    'income_ratio': income_ratio_person.values,
    'weight': income_ratio_person.weights,
    'has_snap': snap_person.values > 0
})
df_person = df_person[df_person['has_snap']].copy()
df_person = df_person.sort_values('income_ratio')

# Calculate cumulative shares for recipients
df_person['cumsum_weight'] = df_person['weight'].cumsum()
df_person['cumsum_pct'] = df_person['cumsum_weight'] / df_person['weight'].sum() * 100
df_person['income_pct'] = df_person['income_ratio'] * 100

# Calculate for benefits (SPM unit level)
snap_spm = sim.calculate('snap', period='2025-10', map_to='spm_unit')
income_ratio_spm = sim.calculate('snap_gross_income_fpg_ratio', period='2025-10', map_to='spm_unit')

# Create dataframe for benefits
df_spm = pd.DataFrame({
    'income_ratio': income_ratio_spm.values,
    'weight': income_ratio_spm.weights,
    'snap': snap_spm.values,
    'has_snap': snap_spm.values > 0
})
df_spm = df_spm[df_spm['has_snap']].copy()
df_spm = df_spm.sort_values('income_ratio')
df_spm['weighted_snap'] = df_spm['snap'] * df_spm['weight']

# Calculate cumulative shares for benefits
df_spm['cumsum_benefits'] = df_spm['weighted_snap'].cumsum()
df_spm['cumsum_pct'] = df_spm['cumsum_benefits'] / df_spm['weighted_snap'].sum() * 100
df_spm['income_pct'] = df_spm['income_ratio'] * 100

In [ ]:
# Create figure
fig = go.Figure()

# Add recipients line (teal)
fig.add_trace(go.Scatter(
    x=df_person['income_pct'],
    y=df_person['cumsum_pct'],
    mode='lines',
    name='Recipients',
    line=dict(color=COLOUR_SCHEMES['teal']['primary'], width=2),
    hovertemplate='%{x:.0f}% FPL<br>%{y:.1f}% of recipients<extra></extra>'
))

# Add benefits line (blue)
fig.add_trace(go.Scatter(
    x=df_spm['income_pct'],
    y=df_spm['cumsum_pct'],
    mode='lines',
    name='Benefits',
    line=dict(color=COLOUR_SCHEMES['blue']['primary'], width=2),
    hovertemplate='%{x:.0f}% FPL<br>%{y:.1f}% of benefits<extra></extra>'
))

# Add vertical lines at 130% and 200%
fig.add_vline(x=130, line_dash="dash", line_color=COLOUR_SCHEMES['gray']['primary'], opacity=0.5)
fig.add_vline(x=200, line_dash="dash", line_color=COLOUR_SCHEMES['gray']['primary'], opacity=0.5)

# Add annotations for the vertical lines
fig.add_annotation(x=130, y=95, text="130% FPL", showarrow=False, 
                   font=dict(size=12, color=COLOUR_SCHEMES['gray']['dark']))
fig.add_annotation(x=200, y=95, text="200% FPL", showarrow=False,
                   font=dict(size=12, color=COLOUR_SCHEMES['gray']['dark']))

# Apply PolicyEngine formatting
fig = format_figure(
    fig,
    title='Cumulative Distribution of SNAP Recipients and Benefits by Gross Income',
    x_title='Gross Income (% of Federal Poverty Level)',
    y_title='Cumulative Share (%)',
    colour_scheme='teal',
    height=500,
    width=800
)

# Add PolicyEngine logo at bottom right
fig.add_layout_image(
    dict(
        source="https://policyengine.org/images/logos/policyengine/blue.png",
        xref="paper",
        yref="paper",
        x=0.98,
        y=0.02,
        sizex=0.15,
        sizey=0.15,
        xanchor="right",
        yanchor="bottom",
        opacity=0.5
    )
)

fig.update_xaxes(range=[0, 300])
fig.update_yaxes(range=[0, 100])

fig.show()

## Key Statistics

In [ ]:
# Calculate summary statistics
snap = sim.calculate('snap', period='2025-10', map_to='person')
income_ratio = sim.calculate('snap_gross_income_fpg_ratio', period='2025-10', map_to='person')

# Recipients above 130%
snap_recipients = snap > 0
above_130_fpl = (snap > 0) & (income_ratio > 1.3)
total_snap_recipients = snap_recipients.sum()
total_above_130 = above_130_fpl.sum()
share_130 = total_above_130 / total_snap_recipients

# Recipients above 200%
above_200_fpl = (snap > 0) & (income_ratio > 2.0)
total_above_200 = above_200_fpl.sum()
share_200 = total_above_200 / total_snap_recipients

# Benefits above 130% and 200%
snap_spm = sim.calculate('snap', period='2025-10', map_to='spm_unit')
spm_gross_income_ratio = sim.calculate('snap_gross_income_fpg_ratio', period='2025-10', map_to='spm_unit')

total_snap_benefits = snap_spm.sum()
snap_above_130_spm = snap_spm * (spm_gross_income_ratio > 1.3)
snap_benefits_above_130 = snap_above_130_spm.sum()
share_benefits_130 = snap_benefits_above_130 / total_snap_benefits

snap_above_200_spm = snap_spm * (spm_gross_income_ratio > 2.0)
snap_benefits_above_200 = snap_above_200_spm.sum()
share_benefits_200 = snap_benefits_above_200 / total_snap_benefits

print("=== Above 130% FPL ===")
print(f"Recipients: {share_130:.2%} ({total_above_130:,.0f} of {total_snap_recipients:,.0f})")
print(f"Benefits: {share_benefits_130:.2%} (${snap_benefits_above_130:,.0f} of ${total_snap_benefits:,.0f})")

print("\n=== Above 200% FPL ===")
print(f"Recipients: {share_200:.2%} ({total_above_200:,.0f} of {total_snap_recipients:,.0f})")
print(f"Benefits: {share_benefits_200:.2%} (${snap_benefits_above_200:,.0f} of ${total_snap_benefits:,.0f})")

=== Above 130% FPL ===
Recipients: 23.86% (11,032,938 of 46,239,099)
Benefits: 11.33% ($834,789,722 of $7,365,073,059)

=== Above 200% FPL ===
Recipients: 2.86% (1,322,911 of 46,239,099)
Benefits: 2.52% ($185,661,097 of $7,365,073,059)


## Analysis: Who Receives SNAP Above 200% FPL?

To understand who qualifies for SNAP with income exceeding 200% FPL (both gross and net), we examine the eligibility pathways.

In [ ]:
# Analyze those above 200% FPL (both gross and net)
net_ratio = sim.calculate('snap_net_income_fpg_ratio', period='2025-10', map_to='person')
has_elderly_disabled = sim.calculate('has_usda_elderly_disabled', period='2025-10', map_to='person')
ssi = sim.calculate('ssi', period='2025-10', map_to='person')

# Filter to those above 200% both gross and net
above_200_both = (snap > 0) & (income_ratio > 2.0) & (net_ratio > 2.0)

total_above_200_both = above_200_both.sum()
with_elderly_disabled = (above_200_both & has_elderly_disabled).sum()
with_ssi = (above_200_both & (ssi > 0)).sum()

# At SPM unit level
net_ratio_spm = sim.calculate('snap_net_income_fpg_ratio', period='2025-10', map_to='spm_unit')
above_200_both_spm = (snap_spm > 0) & (spm_gross_income_ratio > 2.0) & (net_ratio_spm > 2.0)
total_units_above_200_both = above_200_both_spm.sum()

print("=== SNAP Recipients Above 200% FPL (Both Gross and Net Income) ===")
print(f"\nTotal recipients: {total_above_200_both:,.0f}")
print(f"Total SPM units: {total_units_above_200_both:,.0f}")
print(f"\nEligibility pathway:")
print(f"  In SPM units with elderly (60+) or disabled members: {with_elderly_disabled:,.0f} ({with_elderly_disabled/total_above_200_both*100:.2f}%)")
print(f"  Receiving SSI: {with_ssi:,.0f} ({with_ssi/total_above_200_both*100:.2f}%)")
print(f"  ")
print(f"Key finding: 100% of SNAP recipients above 200% FPL (both gross and net income)")
print(f"live in households with at least one elderly or disabled member.")
print(f"\nThe elderly/disabled exemption:")
print(f"  - Exempts households from the gross income test (can exceed 130% FPL or state BBCE limits)")
print(f"  - Still requires passing the net income test (200% FPL after deductions)")
print(f"  - Applies to entire household if any member is 60+ or disabled")

=== SNAP Recipients Above 200% FPL (Both Gross and Net Income) ===

Total recipients: 159,723
Total SPM units: 79,228

Eligibility pathway:
  In SPM units with elderly (60+) or disabled members: 159,723 (100.00%)
  Receiving SSI: 79,228 (49.60%)
  
Key finding: 100% of SNAP recipients above 200% FPL (both gross and net income)
live in households with at least one elderly or disabled member.

The elderly/disabled exemption:
  - Exempts households from the gross income test (can exceed 130% FPL or state BBCE limits)
  - Still requires passing the net income test (200% FPL after deductions)
  - Applies to entire household if any member is 60+ or disabled


## Eligibility Context

SNAP recipients can have gross income above 130% FPL through three pathways:

### 1. Categorical Eligibility (28% of those above 200% gross)
Receipt of certain benefits completely bypasses all income and asset tests:
- **SSI (Supplemental Security Income)**: Federal program for elderly, blind, or disabled with limited income
- **TANF non-cash benefits**: Through Broad-Based Categorical Eligibility (BBCE), states can provide minimal TANF services (like informational materials) to make households categorically eligible
- **State BBCE limits**: Many states set BBCE income limits at 200% FPL (CA, MA, MD, MI, NC, PA, VA, WA, and others)

### 2. Elderly/Disabled Household Exemption (100% of those above 200% both gross and net)
Households with members aged 60+ or disabled:
- **Exempt from gross income test**: Can have income exceeding 130% FPL (or state BBCE limits)
- **Must still pass net income test**: Net income (after deductions) must be ≤200% FPL
- **Applies to entire household**: All members benefit if any one member is elderly or disabled
- **Example**: A 49-year-old working person living with an 80-year-old disabled parent can qualify with high gross income if deductions (medical expenses, shelter costs) bring net income below 200% FPL

### Key Observations
- Benefits are concentrated at lower incomes: While 24% of recipients are above 130% FPL, they receive only 11% of benefits
- The elderly/disabled exemption is the dominant pathway for very high-income SNAP receipt (>200% FPL both gross and net)
- PolicyEngine-US models these rules based on [7 CFR § 273.9](https://www.law.cornell.edu/cfr/text/7/273.9) and state-specific BBCE parameters